# LU factorization

## Basic form

Our goal is to factor $A=LU$, where $L$ is unit lower triangular and $U$ is upper triangular. As the book explains, expressing Gaussian elimination using linear algebra leads to an algorithm. We will derive it differently, using the outer product form of $LU$, similarly to how we found modified Gram-Schmidt. 

Define 

$$A_j = \sum_{k=j}^m \ell_k u_k^*, \qquad j=1,\ldots,m.$$

Note that $A_1=A$. Let's step through a small example. 

In [1]:
using LinearAlgebra

A = rand(1:10,4,4)
A1 = A

4×4 Array{Int64,2}:
 2   1   2  6
 3  10   6  2
 2   8  10  2
 6   1   4  3

Let's look at the first row of $A_1$. We can express this algebraically as 

$$e_1^* A_1 = \sum_{k=1}^m (e_1^* \ell_k) u_k^* = u_1^*,$$

thanks to the structure of $L$. From this identity we get the first row of $U$. Similarly, if we look at the first column of $A_1$, we find

$$A_1 e_1 = \sum_{k=1}^m \ell_k (u_k^*e_1) = U_{11}\ell_1,$$

which gives us the first column of $L$. 

(Just for fun, let's try this using `Rational` numbers.)

In [2]:
U = UpperTriangular(fill(0//1,4,4))
L = LowerTriangular(fill(0//1,4,4))

U[1,:] = A1[1,:]
L[:,1] = A1[:,1]//U[1,1]
display(L)
display(U)

4×4 LowerTriangular{Rational{Int64},Array{Rational{Int64},2}}:
 1//1   ⋅     ⋅     ⋅  
 3//2  0//1   ⋅     ⋅  
 1//1  0//1  0//1   ⋅  
 3//1  0//1  0//1  0//1

4×4 UpperTriangular{Rational{Int64},Array{Rational{Int64},2}}:
 2//1  1//1  2//1  6//1
  ⋅    0//1  0//1  0//1
  ⋅     ⋅    0//1  0//1
  ⋅     ⋅     ⋅    0//1

Now we can construct $A_2=A_1-\ell_1u_1^*$. Note that 

$$e_1^* A_2 = e_1^*A_1 - u_1^* = 0,$$ 

and 

$$A_2e_1 = A_1e_1 - \ell_1 U_{11} = 0.$$

In other words, the first rank-one term $\ell_1 u_1^*$ captures and cancels out the first row and column of the original $A$. 

In [3]:
A2 = A1 - L[:,1]*U[1,:]'

4×4 Array{Rational{Int64},2}:
 0//1   0//1   0//1    0//1
 0//1  17//2   3//1   -7//1
 0//1   7//1   8//1   -4//1
 0//1  -2//1  -2//1  -15//1

Now we move on to take out the second column and row. Observe that 

$$ e_2^*A_2 = \sum_{k=2}^m (e_2^* \ell_k) u_k^* = u_2^*,$$

and 

$$A_2 e_2 = \sum_{k=2}^m \ell_k (u_k^*e_2) = U_{22}\ell_2.$$

In [4]:
U[2,:] = A2[2,:]
L[:,2] = A2[:,2]/U[2,2]
display(L)
display(U)

4×4 LowerTriangular{Rational{Int64},Array{Rational{Int64},2}}:
 1//1    ⋅      ⋅     ⋅  
 3//2   1//1    ⋅     ⋅  
 1//1  14//17  0//1   ⋅  
 3//1  -4//17  0//1  0//1

4×4 UpperTriangular{Rational{Int64},Array{Rational{Int64},2}}:
 2//1   1//1  2//1   6//1
  ⋅    17//2  3//1  -7//1
  ⋅      ⋅    0//1   0//1
  ⋅      ⋅     ⋅     0//1

You may have guessed that this will capture the second row and column, so that we can zero those out too.

In [5]:
A3 = A2 - L[:,2]*U[2,:]'

4×4 Array{Rational{Int64},2}:
 0//1  0//1    0//1      0//1 
 0//1  0//1    0//1      0//1 
 0//1  0//1   94//17    30//17
 0//1  0//1  -22//17  -283//17

And so on.

In [6]:
U[3,:] = A3[3,:]
L[:,3] = A3[:,3]/U[3,3]
A4 = A3 - L[:,3]*U[3,:]'

U[4,:] = A4[4,:]
L[:,4] = A4[:,4]/U[4,4]
display(L)
display(U)

4×4 LowerTriangular{Rational{Int64},Array{Rational{Int64},2}}:
 1//1    ⋅        ⋅      ⋅  
 3//2   1//1      ⋅      ⋅  
 1//1  14//17    1//1    ⋅  
 3//1  -4//17  -11//47  1//1

4×4 UpperTriangular{Rational{Int64},Array{Rational{Int64},2}}:
 2//1   1//1   2//1      6//1 
  ⋅    17//2   3//1     -7//1 
  ⋅      ⋅    94//17    30//17
  ⋅      ⋅      ⋅     -763//47

This is the whole factorization:

In [7]:
A-L*U

4×4 Array{Rational{Int64},2}:
 0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1

Before we can write this as an algorithm, though, we need to address an important possible failure point.

## Pivoting

Above, we iteratively divided by the entries $U_{11},U_{22},\ldots$ as we found them. What if one of these were zero? We can easily find an example where this happens almost immediately.

In [8]:
A[1,1] = 0
A

4×4 Array{Int64,2}:
 0   1   2  6
 3  10   6  2
 2   8  10  2
 6   1   4  3

In [9]:
U = zeros(4,4)
L = zeros(4,4)

A1 = A
U[1,:] = A1[1,:]
L[:,1] = A1[:,1]/U[1,1]
display(L)
display(U)

4×4 Array{Float64,2}:
 NaN  0.0  0.0  0.0
 Inf  0.0  0.0  0.0
 Inf  0.0  0.0  0.0
 Inf  0.0  0.0  0.0

4×4 Array{Float64,2}:
 0.0  1.0  2.0  6.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

You might be thinking that maybe $A$ is singular, so we're off the hook. But that is not the case.

In [10]:
svdvals(A)

4-element Array{Float64,1}:
 18.702392214146194 
  6.501913467263393 
  4.70885263842889  
  3.1260763845472246

However, there's a part of standard Gaussian elimination we have not yet used: swapping rows of the matrix. In a linear system of equations, this leaves the solution unchanged. By swapping rows to put a nonzero in the "pivot" location $(k,k)$ of $A_k$, the algorithm may continue. 

We'll look at this a little differently. Rather than trying to zero out the first row and first column with $\ell_1 u_1^*$, we will chose a different row, which we denote by $i_1$. We will also change the old structural requirements 

$$L_{11} = 1, \quad L_{12}=L_{13}=\cdots=L_{1m}=0,$$ 

to hold for row $i_1$ rather than row 1. So now 

$$e_{i_1}^* A_1 = \sum_{k=1}^m (e_{i_1}^* \ell_k) u_k^* = u_1^*,$$

which as before gives a way to extract $u_1^*$. But now $U_{11}$ is the $(i_1,1)$ element of $A$. If we can't find an $i_1$ such that this is nonzero, then the entire first column of $A$ is zero, and this *would* imply that $A$ is singular. Otherwise, we have $A_1 e_1=U_{11}\ell_1$ exactly as before, and we know that we can compute $\ell_1$. 

This is a lot less daunting than the formalism makes it sound. First, we use the maximum element in column 1 to select $i_1$ (more on this later). 

In [11]:
i = zeros(Int,4);
i[1] = findmax(abs.(A1[:,1]))[2]

4

So we are targeting row 3 and column 1 to zero out. 

In [12]:
A1 = A
U[1,:] = A1[i[1],:]
L[:,1] = A1[:,1]/U[1,1]
display(L)
display(U)

4×4 Array{Float64,2}:
 0.0       0.0  0.0  0.0
 0.5       0.0  0.0  0.0
 0.333333  0.0  0.0  0.0
 1.0       0.0  0.0  0.0

4×4 Array{Float64,2}:
 6.0  1.0  4.0  3.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [13]:
A2 = A1 - L[:,1]*U[1,:]'

4×4 Array{Float64,2}:
 0.0  1.0      2.0      6.0
 0.0  9.5      4.0      0.5
 0.0  7.66667  8.66667  1.0
 0.0  0.0      0.0      0.0

Now we select a new row $i_2$ with a nonzero pivot in column 2.

In [14]:
i[2] = findmax(abs.(A2[:,2]))[2]

2

Now we want $e_{i_2}^*A_2=u_2^*$. This happens if we require

$$L_{i_2,2} = 1, \quad L_{i_2,3}=\cdots=L_{i_2,m}=0.$$ 

(Note that $L_{i_2,1}$ was previously determined.) 

In [15]:
U[2,:] = A2[i[2],:]
L[:,2] = A2[:,2]/U[2,2]
display(L)
display(U)

4×4 Array{Float64,2}:
 0.0       0.105263  0.0  0.0
 0.5       1.0       0.0  0.0
 0.333333  0.807018  0.0  0.0
 1.0       0.0       0.0  0.0

4×4 Array{Float64,2}:
 6.0  1.0  4.0  3.0
 0.0  9.5  4.0  0.5
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [16]:
A3 = A2 - L[:,2]*U[2,:]'

4×4 Array{Float64,2}:
 0.0  0.0  1.57895  5.94737 
 0.0  0.0  0.0      0.0     
 0.0  0.0  5.4386   0.596491
 0.0  0.0  0.0      0.0     

By now the pattern is clear. 

In [17]:
i[3] = findmax(abs.(A3[:,3]))[2]
U[3,:] = A3[i[3],:]
L[:,3] = A3[:,3]/U[3,3]
A4 = A3 - L[:,3]*U[3,:]'

i[4] = findmax(abs.(A4[:,4]))[2]
U[4,:] = A4[i[4],:]
L[:,4] = A4[:,4]/U[4,4];

Indeed, we did again get a factorization of $A$.

In [18]:
norm(A-L*U)

2.220446049250313e-16

But what sort of factorization is it? 

In [19]:
display(L)
display(U)

4×4 Array{Float64,2}:
 0.0       0.105263  0.290323  1.0
 0.5       1.0       0.0       0.0
 0.333333  0.807018  1.0       0.0
 1.0       0.0       0.0       0.0

4×4 Array{Float64,2}:
 6.0  1.0   4.0          3.0     
 0.0  9.5   4.0          0.5     
 0.0  0.0   5.4386       0.596491
 0.0  0.0  -2.22045e-16  5.77419 

Just as before, $U$ is upper triangular. But $L$ is not triangular. However, think about the structural conditions imposed during the algorithm:

$$L_{i_1,1} = 1, \quad L_{1_1,2}=\cdots=L_{i_1,m}=0,$$ 

$$L_{i_2,2} = 1, \quad L_{1_2,3}=\cdots=L_{i_2,m}=0,$$ 

down to $L_{i_m,i_m}=1$ in the last step. What this means is that if we take the rows of $L$ in the order $i_1,i_2,i_3,i_4$, then the result is again unit lower triangular! 

In [20]:
L[i,:]

4×4 Array{Float64,2}:
 1.0       0.0       0.0       0.0
 0.5       1.0       0.0       0.0
 0.333333  0.807018  1.0       0.0
 0.0       0.105263  0.290323  1.0

We can express this result using a permutation matrix as $PL$. Conventionally though, this truly triangular matrix is the one we call $L$, and the one produced directly by the algorithm is $P^{-1}L=P^TL$. Since $A=P^TLU$, this implies that $PA=LU$. This is the **row-pivoted LU factorization** (or partially pivoted factorization, or $P^TLU$ factorization). 

## Linear systems

The system $Ax=b$ is equivalent to $PAx=Pb$, or $L(Ux)=Pb$. We do a forward substitution using a permnuted form of $b$, then a backward substitution using that result. (In practice we wouldn't move data around in memory, but just index the vector indirectly in the correct order.) 

In [21]:
xact = ones(4);  b = A*xact;

Pb = similar(b);  Pb = b[i]; 
x = U\(L[i,:]\Pb)

4-element Array{Float64,1}:
 0.9999999999999999
 1.0               
 1.0000000000000002
 0.9999999999999999

The built-in `lu` returns a factorization object that will show you the (truly triangular) $L$ and $U$ factors, and the permutation vector we called `i` in the example above.

In [22]:
fact = lu(A);
typeof(fact)

LU{Float64,Array{Float64,2}}

In [23]:
fact.L

4×4 Array{Float64,2}:
 1.0       0.0       0.0       0.0
 0.5       1.0       0.0       0.0
 0.333333  0.807018  1.0       0.0
 0.0       0.105263  0.290323  1.0

In [24]:
fact.U

4×4 Array{Float64,2}:
 6.0  1.0  4.0     3.0     
 0.0  9.5  4.0     0.5     
 0.0  0.0  5.4386  0.596491
 0.0  0.0  0.0     5.77419 

In [25]:
fact.p

4-element Array{Int64,1}:
 4
 2
 3
 1